In [52]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import itertools
import tensorflow as tf
import pandas as pd

import tensorflow
from tensorflow.keras import backend as K
from utility import LoadDataIot
from Model_cnn import DCNet_Dropouts   #DCNet_Dropouts
import random
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
import numpy as np
#import tensorflow_addons as tf
import tensorflow
from sklearn.utils import shuffle
#from tensorflow_addons.optimizers import extend_with_decoupled_weight_decay
from tensorflow.keras.models import Model
from tensorflow.python.ops.numpy_ops import np_config
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
np_config.enable_numpy_behavior()


In [53]:
!python dataset/Class_filter.py

2023-09-18 02:54:06.068618: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
256 249 260 253 249 236 251 235 261 259
(3180, 500)
(3180,)
[0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6
 7 8 9 0 1 2 3 4 5 6 7 8 9]
(1272, 500)
Data dimensions:
X: Training data's shape :  (890, 500)
X: Validating data's shape :  (229, 500)
X: Testing data's shape :  (153, 500)
X: open data's shape :  (1908, 500)


In [54]:
NB_EPOCH = 500   # Number of training epoch
print("Number of Epoch: ", NB_EPOCH)
BATCH_SIZE = 64 # Batch size
VERBOSE = 2 # Output display mode
LENGTH = 500 # Packet sequence length
NB_CLASSES = 4 # number of outputs = number of classes
INPUT_SHAPE = (LENGTH,1)
Lambda=1
alpha=1

random.seed(0)

def representative_data_gen():
  for input_value in tensorflow.data.Dataset.from_tensor_slices(X_train_Rep).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

def representative_data_gen_Open():
  for input_value in tf.data.Dataset.from_tensor_slices(X_open).batch(1).take(1000):
    # Model has only one input so each data point has one element.
    yield [input_value]

def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

def convert_bytes(size, unit=None):
    if unit == "KB":
        return str(round(size / 1024, 3))
    elif unit == "MB":
        return str(round(size / (1024 * 1024), 3))
    else:
        return str(size)

def Micro_F1(matrix):
    epsilon = 1e-8
    TP = 0
    FP = 0
    TN = 0

    for k in range(NB_CLASSES):
        TP += matrix[k][k]
        FP += (np.sum(Matrix, axis=0)[k] - matrix[k][k])
        TN += (np.sum(Matrix, axis=1)[k] - matrix[k][k])

    Micro_Prec = TP / (TP + FP)
    Micro_Rec = TP / (TP + TN)
    print("Micro Precision: ", Micro_Prec)
    print("Micro Recall: ", Micro_Rec)
    Micro_F1 = 2 * Micro_Prec * Micro_Rec / (Micro_Rec + Micro_Prec + epsilon)

    return Micro_F1


def New_F1_Score(Matrix):
    Column_sum = np.sum(Matrix, axis=0)
    Raw_sum = np.sum(Matrix, axis=1)

    Precision_Differences = []
    Recall_Differences = []
    for i in range(NB_CLASSES):
        Precision_Differences.append(np.abs(2 * Matrix[i][i] - Column_sum[i]))
        Recall_Differences.append(np.abs(2 * Matrix[i][i] - Raw_sum[i]))

    Precision_Differences = np.array(Precision_Differences)
    Precision_Differences_Per = Precision_Differences / np.sum(Precision_Differences)
    Recall_Differences = np.array(Recall_Differences)
    Recall_Differences_Per = Recall_Differences / np.sum(Recall_Differences)

    # print('Precision_Differences_Per',Precision_Differences_Per)
    # print('Recall_Differences_Per',Recall_Differences_Per)

    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = (Matrix[k][k] / np.sum(Matrix, axis=0)[k])
    Precision = np.sum(np.array(Precisions) * Precision_Differences_Per)

    for k in range(len(Recalls)):
        Recalls[k] = (Matrix[k][k] / np.sum(Matrix, axis=1)[k])  # *Recall_Differences_Per[k]
    Recall = np.sum(np.array(Recalls) * Recall_Differences_Per)

    print('Precision: ', Precision)
    print('Recall: ', Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score


def Macro_F1(Matrix):
    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = Matrix[k][k] / np.sum(Matrix, axis=0)[k]
    # print(Precisions)

    Precision = np.average(Precisions)
    print("Macro Precision:", Precision)

    for k in range(len(Recalls)):
        Recalls[k] = Matrix[k][k] / np.sum(Matrix, axis=1)[k]

    Recall = np.average(Recalls)
    print("Macro Recall:", Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score

OPTIMIZER = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=0.0001) # Optimizer


# Data: shuffled and split between train and test sets
print ("Loading and preparing data for training, and evaluating the model")
X_train, y_train, X_test, y_test, X_valid, y_valid  = LoadDataIot()

# Convert data as float32 type
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_valid = y_valid.astype('float32')
y_test = y_test.astype('float32')
# we need a [Length x 1] x n shape as input to the DF CNN (Tensorflow)
X_train = X_train[:, :,np.newaxis]
X_valid = X_valid[:, :,np.newaxis]
X_test = X_test[:, :,np.newaxis]

X_train_Rep,y_train_Rep=shuffle(X_train, y_train)

y_train = to_categorical(y_train, NB_CLASSES)
y_valid = to_categorical(y_valid, NB_CLASSES)
y_test = to_categorical(y_test, NB_CLASSES)


Number of Epoch:  500
Loading and preparing data for training, and evaluating the model
Loading non-defended dataset for closed-world scenario
Data dimensions:
X: Training data's shape :  (890, 500)
y: Training data's shape :  (890,)
X: Validation data's shape :  (229, 500)
y: Validation data's shape :  (229,)
X: Testing data's shape :  (153, 500)
y: Testing data's shape :  (153,)


In [55]:

# Building and training model
print("Building and training DC model")

model = DCNet_Dropouts.build(input_shape=INPUT_SHAPE, nb_classes=NB_CLASSES)

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
    metrics=["accuracy"])

filepath = '/content/training_history/DC_without_norm.hdf5'

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(X_train, y_train,batch_size=BATCH_SIZE, epochs=NB_EPOCH,verbose=VERBOSE, validation_data=(X_valid, y_valid), callbacks=callbacks_list)

with open('/content/training_history/His_Dict_without_Nom', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
print("############## Training is Done Successfully ###################")

# Start evaluating model with testing data

model_penultimate = Model(model.input, model.layers[-2].output)
model_penultimate.save('/content/trained_models/DC_without_softmax.hdf5')
model_penultimate.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
    metrics=["accuracy"])

score_test = model_penultimate.evaluate(X_test, y_test, verbose=VERBOSE)
print("Testing closed accuracy_without_norm:", score_test[1])


Building and training DC model
Epoch 1/500

Epoch 1: val_accuracy improved from -inf to 0.66812, saving model to /content/training_history/DC_without_norm.hdf5
14/14 - 2s - loss: 89.5744 - accuracy: 0.3281 - val_loss: 5.1185 - val_accuracy: 0.6681 - 2s/epoch - 161ms/step
Epoch 2/500

Epoch 2: val_accuracy improved from 0.66812 to 0.82533, saving model to /content/training_history/DC_without_norm.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


14/14 - 0s - loss: 31.3355 - accuracy: 0.4382 - val_loss: 0.7976 - val_accuracy: 0.8253 - 186ms/epoch - 13ms/step
Epoch 3/500

Epoch 3: val_accuracy improved from 0.82533 to 0.89083, saving model to /content/training_history/DC_without_norm.hdf5
14/14 - 0s - loss: 16.9651 - accuracy: 0.4865 - val_loss: 0.3698 - val_accuracy: 0.8908 - 177ms/epoch - 13ms/step
Epoch 4/500

Epoch 4: val_accuracy improved from 0.89083 to 0.91266, saving model to /content/training_history/DC_without_norm.hdf5
14/14 - 0s - loss: 12.6021 - accuracy: 0.5348 - val_loss: 0.3553 - val_accuracy: 0.9127 - 176ms/epoch - 13ms/step
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.91266
14/14 - 0s - loss: 8.0957 - accuracy: 0.5472 - val_loss: 0.4288 - val_accuracy: 0.8734 - 144ms/epoch - 10ms/step
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.91266
14/14 - 0s - loss: 7.2134 - accuracy: 0.5697 - val_loss: 0.5516 - val_accuracy: 0.7773 - 131ms/epoch - 9ms/step
Epoch 7/500

Epoch 7: val_accuracy did no

############## Training is Done Successfully ###################
5/5 - 0s - loss: 6.2304 - accuracy: 0.9608 - 243ms/epoch - 49ms/step
Testing closed accuracy_without_norm: 0.9607843160629272


In [56]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model_fullint.tflite"

converter = tf.lite.TFLiteConverter.from_keras_model(model_penultimate)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

tflite_model_name = "/content/trained_models/"+TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

print("Original Model Size:",convert_bytes(get_file_size('/content/training_history/DC_without_norm.hdf5'), "KB"),"KB")
print("Quantized model size:",convert_bytes(get_file_size(tflite_model_name), "KB"),"KB")
print()

X_test=np.squeeze(X_test, axis=-1)

interpreter = tf.lite.Interpreter(model_path = tflite_model_name)


input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.resize_tensor_input(input_details[0]['index'], (len(X_test), LENGTH, 1))
interpreter.resize_tensor_input(output_details[0]['index'], (len(X_test), NB_CLASSES))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_scale,input_zero_point  = input_details[0]["quantization"]

for i in range(len(X_test)):
  X_test[i] = X_test[i] / input_scale + input_zero_point


X_test = X_test[:, :,np.newaxis]
X_test = X_test.astype('int8')
interpreter.set_tensor(input_details[0]['index'], X_test)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])

prediction_classes = np.argmax(tflite_model_predictions, axis=1)
y_test=np.argmax(y_test, axis=1)

acc_ = accuracy_score(prediction_classes, y_test)
print('Test accuracy TFLITE model :', acc_)

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Original Model Size: 2223.727 KB
Quantized model size: 189.773 KB

[ 76   7 -35 -34]
(153, 4)
(153,)
Test accuracy TFLITE model : 0.9607843137254902


In [57]:
dataset_dir= "/content/dataset/"
X_train = np.load(dataset_dir+'X_train_5.npy')
X_valid = np.load(dataset_dir+'X_valid_5.npy')
X_test= np.load(dataset_dir+'X_test_5.npy')

X_open = np.load(dataset_dir+'X_open.npy')
y__open = np.load(dataset_dir+'y_open.npy')
y_open=[NB_CLASSES]*len(y__open)
X_open,y_open=shuffle(X_open, y_open)

X_open = X_open.astype('float32')

X_valid=np.concatenate((X_valid,X_train[-200:]),axis=0)
y_valid=np.concatenate((y_valid,y_train[-200:]),axis=0)

X_train=X_train[:-200]
y_train=y_train[:-200]

for i in range(NB_CLASSES):
  variable_name = f"Mean_{i}"
  locals()[variable_name]=np.array([0] * NB_CLASSES)


for i in range(len(X_train)):
  X_train[i] = X_train[i] / input_scale + input_zero_point

X_train = X_train[:, :,np.newaxis]
X_train = X_train.astype('int8')

interpreter.resize_tensor_input(input_details[0]['index'], (len(X_train), LENGTH, 1))
interpreter.resize_tensor_input(output_details[0]['index'], (len(X_train), NB_CLASSES))
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], X_train)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])

y_train=np.argmax(y_train,axis=1)
for i in range(len(tflite_model_predictions)):
  if (y_train[i]==0 and np.argmax(tflite_model_predictions[i])==y_train[i]):
    Mean_0=np.sum([Mean_0,tflite_model_predictions[i]], axis=0)

  elif (y_train[i]==1 and np.argmax(tflite_model_predictions[i])==y_train[i]):
    Mean_1=np.sum([Mean_1,tflite_model_predictions[i]], axis=0)

  elif (y_train[i]==2 and np.argmax(tflite_model_predictions[i])==y_train[i]):
    Mean_2=np.sum([Mean_2,tflite_model_predictions[i]], axis=0)

  elif (y_train[i]==3 and np.argmax(tflite_model_predictions[i])==y_train[i]):
    Mean_3=np.sum([Mean_3,tflite_model_predictions[i]], axis=0)

  elif (y_train[i]==4 and np.argmax(tflite_model_predictions[i])==y_train[i]):
    Mean_4=np.sum([Mean_4,tflite_model_predictions[i]], axis=0)

  elif (y_train[i]==5 and np.argmax(tflite_model_predictions[i])==y_train[i]):
    Mean_5=np.sum([Mean_5,tflite_model_predictions[i]], axis=0)

for i in range(NB_CLASSES):
  variable_name = f"Mean_{i}"
  locals()[variable_name] = locals()[variable_name]/list(y_train).count(i)


Mean_vectors = np.array([Mean_0, Mean_1, Mean_2, Mean_3])
np.save('/content/temp_variables/Mean_vectors.npy', Mean_vectors, allow_pickle=True)

interpreter = tf.lite.Interpreter(model_path = tflite_model_name)

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

for i in range(len(X_valid)):
  X_valid[i] = X_valid[i] / input_scale + input_zero_point

interpreter.resize_tensor_input(input_details[0]['index'], (len(X_valid), 500, 1))
interpreter.resize_tensor_input(output_details[0]['index'], (len(X_valid), NB_CLASSES))
interpreter.allocate_tensors()

input_scale,input_zero_point  = input_details[0]["quantization"]

X_valid = X_valid[:, :,np.newaxis]
X_valid = X_valid.astype('int8')

interpreter.set_tensor(input_details[0]['index'], X_valid)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])


In [58]:
Dis_Values={}

for i in range(NB_CLASSES):
  Dis_Values[i]=[]

y_valid=np.argmax(y_valid,axis=1)

for i in range(len(tflite_model_predictions)):
    if (y_valid[i]==0 and np.argmax(tflite_model_predictions[i])==y_valid[i]):
      dist = np.linalg.norm(Mean_0-tflite_model_predictions[i])
      Dis_Values[0].append(dist)

    elif (y_valid[i]==1 and np.argmax(tflite_model_predictions[i])==y_valid[i]):
      dist = np.linalg.norm(Mean_1-tflite_model_predictions[i])
      Dis_Values[1].append(dist)

    elif (y_valid[i]==2 and np.argmax(tflite_model_predictions[i])==y_valid[i]):
      dist = np.linalg.norm(Mean_2-tflite_model_predictions[i])
      Dis_Values[2].append(dist)

    elif (y_valid[i]==3 and np.argmax(tflite_model_predictions[i])==y_valid[i]):
      dist = np.linalg.norm(Mean_3-tflite_model_predictions[i])
      Dis_Values[3].append(dist)

    elif (y_valid[i]==4 and np.argmax(tflite_model_predictions[i])==y_valid[i]):
      dist = np.linalg.norm(Mean_4-tflite_model_predictions[i])
      Dis_Values[4].append(dist)

    elif (y_valid[i]==5 and np.argmax(tflite_model_predictions[i])==y_valid[i]):
      dist = np.linalg.norm(Mean_5-tflite_model_predictions[i])
      Dis_Values[5].append(dist)



percentile = 0.9
for i in range(NB_CLASSES):
    Dis_Values[i].sort()
    variable_name = f"Th_{i}"
    locals() [variable_name] = Dis_Values[i][int(len(Dis_Values[i]) * percentile)]

Threasholds_1 = np.array([Th_0, Th_1, Th_2, Th_3])
np.save('/content/temp_variables/Threasholds_1.npy', Threasholds_1)

print("Thresholds are calulated for method 1")

############################################################################################
Dis_Values={}

for i in range(NB_CLASSES):
  Dis_Values[i]=[]


for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==0):
    dist=np.linalg.norm(Mean_vectors[0]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=0:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions[i])-dist
    Dis_Values[0].append(Tot)


  elif (y_valid[i]==1):
    dist=np.linalg.norm(Mean_vectors[1]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=1:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions[i])-dist
    Dis_Values[1].append(Tot)

  elif (y_valid[i]==2):
    dist=np.linalg.norm(Mean_vectors[2]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=2:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions[i])-dist
    Dis_Values[2].append(Tot)

  elif (y_valid[i]==3):
    dist=np.linalg.norm(Mean_vectors[3]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=3:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions[i])-dist
    Dis_Values[3].append(Tot)

percentile = 0.9
for i in range(NB_CLASSES):
    Dis_Values[i].sort()
    variable_name = f"Th_{i}"
    locals() [variable_name] = Dis_Values[i][int(len(Dis_Values[i]) * percentile)]

Threasholds_2 = np.array([Th_0, Th_1, Th_2, Th_3])
np.save('/content/temp_variables/Threasholds_2.npy', Threasholds_2)

print("Thresholds are calulated for method 2")

############################################################################################
Dis_Values={}

for i in range(NB_CLASSES):
  Dis_Values[i]=[]


for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==0):
    dist=np.linalg.norm(Mean_vectors[0]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=0:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions[i])
    Dis_Values[0].append(dist/Tot)


  elif (y_valid[i]==1):
    dist=np.linalg.norm(Mean_vectors[1]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=1:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions[i])
    Dis_Values[1].append(dist/Tot)

  elif (y_valid[i]==2):
    dist=np.linalg.norm(Mean_vectors[2]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=2:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions[i])
    Dis_Values[2].append(dist/Tot)

  elif (y_valid[i]==3):
    dist=np.linalg.norm(Mean_vectors[3]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=3:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions[i])
    Dis_Values[3].append(dist/Tot)


percentile = 0.9
for i in range(NB_CLASSES):
    Dis_Values[i].sort()
    variable_name = f"Th_{i}"
    locals() [variable_name] = Dis_Values[i][int(len(Dis_Values[i]) * percentile)]

Threasholds_3 = np.array([Th_0, Th_1, Th_2, Th_3])
np.save('/content/temp_variables/Threasholds_3.npy', Threasholds_3)

print("Thresholds are calulated for method 3")


Thresholds are calulated for method 1
Thresholds are calulated for method 2
Thresholds are calulated for method 3


In [59]:
interpreter = tf.lite.Interpreter(model_path = '/content/trained_models/tf_lite_model_fullint.tflite')
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.resize_tensor_input(input_details[0]['index'], (len(X_test), 500, 1))
interpreter.resize_tensor_input(output_details[0]['index'], (len(X_test), NB_CLASSES))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


for i in range(len(X_test)):
  X_test[i] = X_test[i] / input_scale + input_zero_point
for i in range(len(X_open)):
  X_open[i] = X_open[i] / input_scale + input_zero_point

# #print(X_test[0])
X_test = X_test[:, :,np.newaxis]
X_test = X_test.astype('int8')
X_open = X_open[:, :,np.newaxis]
X_open = X_open.astype('int8')

interpreter.set_tensor(input_details[0]['index'], X_test)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])

#print("######### Closed set ##############")
#print(tflite_model_predictions[:5])

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.resize_tensor_input(input_details[0]['index'], (len(X_open), LENGTH, 1))
interpreter.resize_tensor_input(output_details[0]['index'], (len(X_open), NB_CLASSES))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], X_open)
interpreter.invoke()
tflite_model_predictions_open = interpreter.get_tensor(output_details[0]['index'])


#############################################################################################################

In [60]:
print()
print("############## Distance Method 1 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions)):

    d=np.argmax(tflite_model_predictions[i], axis=0)
    if np.linalg.norm(tflite_model_predictions[i]-Mean_vectors[d])>Threasholds_1[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes=np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test)
print('Test accuracy Normal model_Closed_set :', acc_Close)

tflite_model_predictions_open = model_penultimate.predict(X_open)
prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):

    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    if np.linalg.norm(tflite_model_predictions_open[i]-Mean_vectors[d])>Threasholds_1[d]:
      prediction_classes_open.append(NB_CLASSES)

    else:
      prediction_classes_open.append(d)

prediction_classes_open=np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open)
print('Test accuracy Normal model_Open_set :', acc_Open)

y_test=y_test[:len(prediction_classes)]
y_open=y_open[:len(prediction_classes_open)]

Matrix=[]
for i in range(NB_CLASSES+1):
  Matrix.append(np.zeros(NB_CLASSES+1))


for i in range(len(y_test)):
  Matrix[y_test[i]][prediction_classes[i]]+=1

for i in range(len(y_open)):
  Matrix[y_open[i]][prediction_classes_open[i]]+=1

F1_Score=New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
print("Micro")
#print('Test accuracy TFLITE model_Closed_set :', acc_Close)
#print('Test accuracy TFLITE model_Open_set :', acc_Open)
F1_Score=Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
print("Macro")
#print('Test accuracy TFLITE model_Closed_set :', acc_Close)
#print('Test accuracy TFLITE model_Open_set :', acc_Open)
F1_Score=Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)




#############################################################################################################
print()
print("############## Distance Method 2 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions)):
    d=np.argmax(tflite_model_predictions[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions[i])-dist

    if Tot<Threasholds_2[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes=np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test)
print('Test accuracy Normal model_Closed_set :', acc_Close)

tflite_model_predictions_open = model.predict(X_open)
prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):
    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_open[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_open[i])-dist

    if Tot<Threasholds_2[d]:
      prediction_classes_open.append(NB_CLASSES)

    else:
      prediction_classes_open.append(d)

prediction_classes_open=np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open)
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix=[]
for i in range(NB_CLASSES+1):
  Matrix.append(np.zeros(NB_CLASSES+1))


for i in range(len(y_test)):
  Matrix[y_test[i]][prediction_classes[i]]+=1

for i in range(len(y_open)):
  Matrix[y_open[i]][prediction_classes_open[i]]+=1

F1_Score=New_F1_Score(Matrix)

print("Average Novel F1_Score: ", F1_Score)

print()
print("Micro")
F1_Score=Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
print("Macro")
F1_Score=Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)



#############################################################################################################
print()
print("############## Distance Method 3 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions)):
    d=np.argmax(tflite_model_predictions[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions[i])

    Tot=dist/Tot
    if Tot>Threasholds_3[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes=np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test)
print('Test accuracy Normal model_Closed_set :', acc_Close)

tflite_model_predictions_open = model.predict(X_open)
prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):
    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_open[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_open[i])

    Tot=dist/Tot
    if Tot>Threasholds_3[d]:
      prediction_classes_open.append(NB_CLASSES)

    else:
      prediction_classes_open.append(d)

prediction_classes_open=np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open)
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix=[]
for i in range(NB_CLASSES+1):
  Matrix.append(np.zeros(NB_CLASSES+1))


for i in range(len(y_test)):
  Matrix[y_test[i]][prediction_classes[i]]+=1

for i in range(len(y_open)):
  Matrix[y_open[i]][prediction_classes_open[i]]+=1

F1_Score=New_F1_Score(Matrix)

print("Average Novel F1_Score: ", F1_Score)

print()
print("Micro")
F1_Score=Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
print("Macro")
F1_Score=Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)


############## Distance Method 1 #################################

Test accuracy Normal model_Closed_set : 0.8823529411764706
60/60 [==============================] - 0s 2ms/step
Test accuracy Normal model_Open_set : 1.0
[array([35.,  0.,  0.,  0.,  5.]), array([ 0., 31.,  0.,  0.,  2.]), array([ 0.,  0., 32.,  0.,  6.]), array([ 0.,  0.,  0., 37.,  5.]), array([   0.,    0.,    0.,    0., 1908.])]
Precision:  1.0
Recall:  0.8852789510684248
Average novel F1_Score:  0.9391490271198059

Micro
Micro Precision:  1.0
Micro Recall:  0.8823529411764706
Average Micro F1_Score:  0.9374999950195313

Macro
Precision: 1.0
Recall: 0.8843628958760538
Average Macro F1_Score:  0.938633310089393

############## Distance Method 2 #################################

Test accuracy Normal model_Closed_set : 0.11764705882352941
60/60 [==============================] - 0s 3ms/step
Test accuracy Normal model_Open_set : 1.0
[array([ 8.,  0.,  0.,  0., 32.]), array([ 0.,  2.,  0.,  0., 31.]), array([ 0.,  0.,